# **Template OP on salish**

In [66]:
%matplotlib inline

import sys
import xarray as xr
import numpy as np
import os
import math
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ErrorCode, ParcelsRandom

sys.path.append('/home/jvalenti/MOAD/analysis-jose/notebooks/parcels')
from Kernels_biofilm import DeleteParticle, Buoyancy, AdvectionRK4_3D, Stokes_drift, Beaching, Unbeaching
from OP_functions_biofilm import *

In [67]:
# Define paths
local = 0 #Set to 0 when working on server
paths = path(local)

In [68]:
path_NEMO = make_prefix(datetime(2018, 1, 10), paths['NEMO'])
coord=xr.open_dataset(paths['coords'],decode_times=False)
WW3 = xr.open_dataset(get_WW3_path(datetime(2018, 1, 10)))
batt=xr.open_dataset(paths['mask'],decode_times=False)

## Define and save mask for distance to coast

In [69]:
Dat=xr.open_dataset(paths['coords'],decode_times=False)
outf_lat=Dat['nav_lat'][445,304]
outf_lon=Dat['nav_lon'][445,304]
clon, clat = [float(outf_lon)],[float(outf_lat)]  # choose horizontal centre of the particle cloud
# #clat,clon = p_unidist(coord.gphif[0,:,:],coord.glamf[0,:,:],batt.mbathy[0,:,:],10,10)
# with open('clat.txt') as f:
#     clat = f.read()
#     clat= clat[1:-1]
#     clat0 = clat.split(",")
#     f.close()
# with open('clon.txt') as f:
#     clon = f.read()
#     clon=clon[1:-1]
#     clon0 = clon.split(",")
#     f.close()
    
# clat,clon=[],[]
# for i in range(len(clat0)):
#     clat.append(float(clat0[i]))
#     clon.append(float(clon0[i]))

## Definitions

In [70]:
start = datetime(2018, 6, 11) #Start date
# Set Time length [days] and timestep [seconds]
length = 45
duration = timedelta(days=length)
dt = 90 #toggle between - or + to pick backwards or forwards
N = 1 # number of deploying locations
n = 500 # 1000   # number of particles per location
dmin = 68 #minimum depth
dd = 5 #max depth difference from dmin
x_offset, y_offset, zvals = p_deploy(N,n,dmin,dd)

In [71]:
from parcels import Variable

class MPParticle(JITParticle):        
    ro = Variable('ro', initial = 1025)           
    diameter = Variable('diameter', initial = 1.6e-5)
    length = Variable('length', initial = 61e-5)
    Lb = Variable('Lb', initial = 0.23)  #days needed in days for particle to have 67% probability of beaching if in beaching zone (500m)
    Db = Variable('Db', initial = 33) #Distance at which particles can randomly beach.
    Ub = Variable('Ub', initial = 69)  #days to have 67% probability of unbeaching
    sediment = Variable('sediment', initial = 0)
    beached = Variable('beached', initial = 0)


In [72]:
lon = np.zeros([N,n])
lat = np.zeros([N,n])
for i in range(N):
    lon[i,:]=(clon[i] + x_offset[i,:])
    lat[i,:]=(clat[i] + y_offset[i,:])
z = zvals

In [73]:
#Set start date time and the name of the output file
name = 'f9Summer' #name output file
daterange = [start+timedelta(days=i) for i in range(length)]
fn =  name + '_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.nc'
outfile = os.path.join(paths['out'], fn)
print(outfile)

/home/jvalenti/MOAD/results/f9Summer20180611_1n_20180726_1n.nc


## Simulation

In [74]:
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W','R']
filenames,variables,dimensions=filename_set(start,length,varlist,local)
field_set=FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)

varlist=['US','VS','WL']
filenames,variables,dimensions=filename_set(start,length,varlist,local)

us = Field.from_netcdf(filenames['US'], variables['US'], dimensions,allow_time_extrapolation=True)
vs = Field.from_netcdf(filenames['VS'], variables['VS'], dimensions,allow_time_extrapolation=True)
wl = Field.from_netcdf(filenames['WL'], variables['WL'], dimensions,allow_time_extrapolation=True)
field_set.add_field(us)
field_set.add_field(vs)
field_set.add_field(wl)
field_set.add_vector_field(VectorField("stokes", us, vs, wl))

filenames,variables,dimensions=filename_set(start,length,['Bathy'],local)
Bth = Field.from_netcdf(filenames['Bathy'], variables['Bathy'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Bth)

In [75]:
# #Load Salish output as fields
#field_set = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)
pset = ParticleSet.from_list(field_set, MPParticle, lon=lon, lat=lat, depth=z, time=start+timedelta(hours=2))

k_sink = pset.Kernel(Buoyancy)
k_waves = pset.Kernel(Stokes_drift)
k_beach = pset.Kernel(Beaching)
k_unbeach = pset.Kernel(Unbeaching)


pset.execute(AdvectionRK4_3D + k_sink + k_waves + k_beach + k_unbeach,
             runtime=duration, 
             dt=dt,
             output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=1)),
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

INFO: Compiled ArrayMPParticleAdvectionRK4_3DBuoyancyStokes_driftBeachingUnbeaching ==> /tmp/parcels-2894/liba98f2e7e072eec0e9d3308c87ebe402b_0.so
INFO: Temporary output files are stored in /home/jvalenti/MOAD/results/out-DFKUCMLV.
INFO: You can use "parcels_convert_npydir_to_netcdf /home/jvalenti/MOAD/results/out-DFKUCMLV" to convert these to a NetCDF file during the run.
N/A% (0 of 3888000.0) |                  | Elapsed Time: 0:00:00 ETA:  --:--:--
  0% (10800.0 of 3888000.0) |            | Elapsed Time: 0:00:00 ETA:   0:02:16
  0% (12600.0 of 3888000.0) |            | Elapsed Time: 0:00:03 ETA:   2:01:33
  0% (14400.0 of 3888000.0) |            | Elapsed Time: 0:00:04 ETA:   1:07:21
  0% (16200.0 of 3888000.0) |            | Elapsed Time: 0:00:07 ETA:   2:05:48
  0% (18000.0 of 3888000.0) |            | Elapsed Time: 0:00:08 ETA:   1:09:29
  0% (19800.0 of 3888000.0) |            | Elapsed Time: 0:00:11 ETA:   1:52:12
  0% (21600.0 of 3888000.0) |            | Elapsed Time: 0:00:11

Correct cell not found for (-124.743065, 48.455124) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 404 0
            new particle indices: (yi, xi) 404 42
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) -4.3014253278443249e+01 4.2687601724935198e-01
Particle 1217 lost !! [2635740.0, 6.938575199507909, 48.455102248469295, -124.74228477246062]


 67% (2631600.0 of 3888000.0) |######    | Elapsed Time: 0:41:17 ETA:   0:17:22
 67% (2633400.0 of 3888000.0) |######    | Elapsed Time: 0:41:20 ETA:   0:40:45
 67% (2635200.0 of 3888000.0) |######    | Elapsed Time: 0:41:21 ETA:   0:22:37
 67% (2637000.0 of 3888000.0) |######    | Elapsed Time: 0:41:24 ETA:   0:42:54
 67% (2638800.0 of 3888000.0) |######    | Elapsed Time: 0:41:25 ETA:   0:23:32
 67% (2640600.0 of 3888000.0) |######    | Elapsed Time: 0:41:28 ETA:   0:39:00
 67% (2642400.0 of 3888000.0) |######    | Elapsed Time: 0:41:28 ETA:   0:21:09
 68% (2644200.0 of 3888000.0) |######    | Elapsed Time: 0:41:31 ETA:   0:28:38
 68% (2646000.0 of 3888000.0) |######    | Elapsed Time: 0:41:31 ETA:   0:15:58
 68% (2647800.0 of 3888000.0) |######    | Elapsed Time: 0:41:34 ETA:   0:31:20
 68% (2649600.0 of 3888000.0) |######    | Elapsed Time: 0:41:34 ETA:   0:17:45
 68% (2651400.0 of 3888000.0) |######    | Elapsed Time: 0:41:38 ETA:   0:39:59
 68% (2653200.0 of 3888000.0) |######   

Correct cell not found for (-124.798109, 48.520760) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 420 0
            new particle indices: (yi, xi) 420 42
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) -4.3044297421293201e+01 9.0256368707827228e-01
Particle 1384 lost !! [2726460.0, 4.477564662659091, 48.520875578014994, -124.79745549417429]


 70% (2721600.0 of 3888000.0) |#######   | Elapsed Time: 0:42:43 ETA:   0:21:32
 70% (2723400.0 of 3888000.0) |#######   | Elapsed Time: 0:42:47 ETA:   0:36:24
 70% (2725200.0 of 3888000.0) |#######   | Elapsed Time: 0:42:47 ETA:   0:20:14
 70% (2727000.0 of 3888000.0) |#######   | Elapsed Time: 0:42:51 ETA:   0:35:31
 70% (2728800.0 of 3888000.0) |#######   | Elapsed Time: 0:42:51 ETA:   0:19:25
 70% (2730600.0 of 3888000.0) |#######   | Elapsed Time: 0:42:53 ETA:   0:26:50
 70% (2732400.0 of 3888000.0) |#######   | Elapsed Time: 0:42:54 ETA:   0:15:02
 70% (2734200.0 of 3888000.0) |#######   | Elapsed Time: 0:42:57 ETA:   0:35:06
 70% (2736000.0 of 3888000.0) |#######   | Elapsed Time: 0:42:57 ETA:   0:19:29
 70% (2737800.0 of 3888000.0) |#######   | Elapsed Time: 0:43:00 ETA:   0:32:04
 70% (2739600.0 of 3888000.0) |#######   | Elapsed Time: 0:43:01 ETA:   0:17:36
 70% (2741400.0 of 3888000.0) |#######   | Elapsed Time: 0:43:03 ETA:   0:26:28
 70% (2743200.0 of 3888000.0) |#######  

...

 91% (3549600.0 of 3888000.0) |######### | Elapsed Time: 0:55:56 ETA:   0:05:33
 91% (3551400.0 of 3888000.0) |######### | Elapsed Time: 0:56:00 ETA:   0:09:49
 91% (3553200.0 of 3888000.0) |######### | Elapsed Time: 0:56:00 ETA:   0:05:21
 91% (3555000.0 of 3888000.0) |######### | Elapsed Time: 0:56:02 ETA:   0:07:54
 91% (3556800.0 of 3888000.0) |######### | Elapsed Time: 0:56:03 ETA:   0:04:23
 91% (3558600.0 of 3888000.0) |######### | Elapsed Time: 0:56:06 ETA:   0:10:23
 91% (3560400.0 of 3888000.0) |######### | Elapsed Time: 0:56:07 ETA:   0:05:44
 91% (3562200.0 of 3888000.0) |######### | Elapsed Time: 0:56:09 ETA:   0:08:30
 91% (3564000.0 of 3888000.0) |######### | Elapsed Time: 0:56:10 ETA:   0:04:40
 91% (3565800.0 of 3888000.0) |######### | Elapsed Time: 0:56:12 ETA:   0:07:39
 91% (3567600.0 of 3888000.0) |######### | Elapsed Time: 0:56:13 ETA:   0:04:14
 91% (3569400.0 of 3888000.0) |######### | Elapsed Time: 0:56:15 ETA:   0:07:12
 91% (3571200.0 of 3888000.0) |#########

Correct cell not found for (-124.767695, 48.484540) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 411 0
            new particle indices: (yi, xi) 411 42
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) -4.3024830811269972e+01 8.1341421828560723e-01
Particle 1011 lost !! [3668760.0, 10.317222239249418, 48.48450894279911, -124.76692607974343]


 94% (3664800.0 of 3888000.0) |######### | Elapsed Time: 0:57:46 ETA:   0:03:44
 94% (3666600.0 of 3888000.0) |######### | Elapsed Time: 0:57:49 ETA:   0:05:08
 94% (3668400.0 of 3888000.0) |######### | Elapsed Time: 0:57:49 ETA:   0:02:50
 94% (3670200.0 of 3888000.0) |######### | Elapsed Time: 0:57:52 ETA:   0:05:09


Correct cell not found for (-124.791014, 48.512304) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 418 0
            new particle indices: (yi, xi) 418 42
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) -4.3035643093140898e+01 7.7828044772845362e-01
Particle 1111 lost !! [3676860.0, 16.049677062482644, 48.51223151012894, -124.79020006345972]


 94% (3672000.0 of 3888000.0) |######### | Elapsed Time: 0:57:52 ETA:   0:03:04
 94% (3673800.0 of 3888000.0) |######### | Elapsed Time: 0:57:56 ETA:   0:06:30
 94% (3675600.0 of 3888000.0) |######### | Elapsed Time: 0:57:56 ETA:   0:03:35
 94% (3677400.0 of 3888000.0) |######### | Elapsed Time: 0:57:59 ETA:   0:06:27
 94% (3679200.0 of 3888000.0) |######### | Elapsed Time: 0:58:00 ETA:   0:03:33
 94% (3681000.0 of 3888000.0) |######### | Elapsed Time: 0:58:02 ETA:   0:04:42
 94% (3682800.0 of 3888000.0) |######### | Elapsed Time: 0:58:02 ETA:   0:02:36
 94% (3684600.0 of 3888000.0) |######### | Elapsed Time: 0:58:05 ETA:   0:04:19
 94% (3686400.0 of 3888000.0) |######### | Elapsed Time: 0:58:05 ETA:   0:02:24
 94% (3688200.0 of 3888000.0) |######### | Elapsed Time: 0:58:08 ETA:   0:05:36
 94% (3690000.0 of 3888000.0) |######### | Elapsed Time: 0:58:08 ETA:   0:03:07
 94% (3691800.0 of 3888000.0) |######### | Elapsed Time: 0:58:12 ETA:   0:06:27
 95% (3693600.0 of 3888000.0) |#########